In [1]:
import pandas as pd
import tensorflow as tf
import numpy as np
np.random.seed(0)
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from tensorflow.keras import Model,Input
from tensorflow.keras.layers import LSTM, Embedding, Dense
from tensorflow.keras.layers import TimeDistributed, SpatialDropout1D,Bidirectional
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from livelossplot.tf_keras import PlotLossesCallback
import gc
gc.collect(),gc.collect()

plt.style.use('fivethirtyeight')

In [4]:
df = pd.read_csv('NER dataset.csv',encoding='latin1')
words = list(set(df['Word'].values))
words.append('ENDPAD')
num_words = len(words)
tags = list(set(df['Tag'].values))
num_tags = len(tags)

In [5]:
class SentenceGetter(object):
    def __init__(self,data):
        self.n_sent = 1
        self.data = data
        agg_func = lambda s:[(w,p,t) for w,p,t in zip(s['Word'].values.tolist(),
                                                     s['POS'].values.tolist(),
                                                     s['Tag'].values.tolist())]
        self.grouped = self.data.groupby("Sentence #").apply(agg_func)
        self.sentences = [s for s in self.grouped]

In [6]:
getter = SentenceGetter(df)
sentences = getter.sentences

In [7]:
word2idx = {w: i+1 for i,w in enumerate(words)}
tag2idx = {t: i for i,t, in enumerate(tags)}

In [8]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

max_len = 50

X = [[word2idx[w[0]] for w in s] for s in sentences]
X = pad_sequences(maxlen=max_len, sequences = X, padding='post', value=num_words-1)

y = [[tag2idx[w[2]] for w in s] for s in sentences]
y = pad_sequences(maxlen=max_len, sequences = y, padding='post', value=tag2idx["O"])
y = [to_categorical(i, num_classes=num_tags) for i in y]

In [9]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.1,random_state=1)

In [10]:
input_word = Input(shape=(max_len,))
model = Embedding(input_dim=num_words, output_dim=max_len, input_length=max_len)(input_word)
model = SpatialDropout1D(0.1)(model)
model = Bidirectional(LSTM(units=100,return_sequences=True,recurrent_dropout=0.1))(model)
out = TimeDistributed(Dense(num_tags, activation='softmax'))(model)
model = Model(input_word, out)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 50)]              0         
                                                                 
 embedding (Embedding)       (None, 50, 50)            1758950   
                                                                 
 spatial_dropout1d (Spatial  (None, 50, 50)            0         
 Dropout1D)                                                      
                                                                 
 bidirectional (Bidirection  (None, 50, 200)           120800    
 al)                                                             
                                                                 
 time_distributed (TimeDist  (None, 50, 17)            3417      
 ributed)                                                        
                                                             

In [11]:
model.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['accuracy'])

In [12]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
early_stopping = EarlyStopping(monitor='val_accuracy', patience=2,verbose=0, mode='max', restore_best_weights = False)
callbacks = [early_stopping]

history = model.fit(
    X_train,np.array(y_train),
    validation_split=0.2,
    batch_size = 64,
    epochs = 5,
    verbose=1
)

Epoch 1/5
540/540 [==============================] - 58s 100ms/step - loss: 0.0645 - accuracy: 0.9937 - val_loss: 0.0108 - val_accuracy: 0.9960
Epoch 2/5
540/540 [==============================] - 57s 105ms/step - loss: 0.0080 - accuracy: 0.9976 - val_loss: 0.0063 - val_accuracy: 0.9981
Epoch 3/5
540/540 [==============================] - 58s 107ms/step - loss: 0.0049 - accuracy: 0.9986 - val_loss: 0.0051 - val_accuracy: 0.9985
Epoch 4/5
540/540 [==============================] - 59s 109ms/step - loss: 0.0037 - accuracy: 0.9990 - val_loss: 0.0047 - val_accuracy: 0.9985
Epoch 5/5
540/540 [==============================] - 58s 108ms/step - loss: 0.0032 - accuracy: 0.9990 - val_loss: 0.0047 - val_accuracy: 0.9985
